### First Regression (Turnover intention ~ unfair treatment x neg. reciprocity)

In [49]:
import pandas as pd
import numpy as np
import seaborn as sns
import math

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

### 1.  Read in SOEP Data:
- vp : 2005 data : main variables of interest: questions on negative reciprocity
- wp: 2006 data : main variables of interest: question on perceived recognition for work
- xp: 2007 data : main variables of interest: turnover intentions, controls

In [50]:
# define path: insert the path where the SOEP data is stored on your computer here
from pathlib import Path
# Path Max: 
# Path Maxie: /Volumes/dohmen_soep/SOEP-CORE.v36eu_STATA/Stata/raw

data_folder = Path("C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/SOEP-Data/Stata/raw")
# define relevant subsets of SOEP-data
file_names = ['vp', 'wp', 'xp']

file_paths = [data_folder / f"{file_name}.dta" for file_name in file_names]
# some controls are in gen data
file_paths_2 = [data_folder / f"{file_name}gen.dta" for file_name in file_names]

In [51]:
# read in 2005 data for the reciprocity measures
data05 = pd.read_stata(file_paths[0], columns=["pid","hid", "syear","vp12602", "vp12603", "vp12605"]).set_index(['pid', 'hid'])
df_2005 = data05.rename(columns={ 'vp12602': 'take_revenge', 'vp12603': 'similar_problems', 'vp12605': 'insult_back'})
# create dummies for take_revenge question
# Create dummy variables
dummies = pd.get_dummies(df_2005['similar_problems'])

# Join the dummy variables to the original dataframe
df_2005 = pd.concat([df_2005, dummies], axis=1)
# rename dummy
df_05 = df_2005.rename(columns = {'[1] Trifft ueberhaupt nicht zu' : 'rec1' , '[2] Skala 1-7' : 'rec2' , '[3] Skala 1-7' : 'rec3' ,'[4] Skala 1-7' : 'rec4' ,'[5] Skala 1-7' : 'rec5' ,'[6] Skala 1-7' : 'rec6' ,'[7] Trifft voll zu' : 'rec7'})
df_05 = df_05.drop(columns = ["[-1] keine Angabe"])

In [52]:
# read in 2006 data
# personal = personal advancement
# still includes all unfair treat
data06 = pd.read_stata(file_paths[1], columns=["pid", "hid", "syear", 'wp43b01', 'wp43b02', 'wp43b03', 'wp43b04', 'wp43b05', 'wp43b06', 'wp43b07','wp43b08', 'wp43a11']).set_index(['pid', 'hid'])
df_06 = data06.rename(columns={ 'wp43b01': 'recog_sup', 'wp43b02': 'felt_recog_sup',"wp43b03": "recog_effort",  'wp43b04': 'felt_recog_effort', "wp43b05": "recog_personal", "wp43b06" :"felt_recog_personal" ,"wp43b07": "recog_pay",'wp43b08': 'felt_recog_pay' , 'wp43a11': 'jobatrisk'})

In [53]:
#read in 2007 data
# here left out 'xp8601' for school degree since we have it in another module also 'xp0102' : 'work_satisfaction' for the beginning
#for outcome and all controls
data3= pd.read_stata(file_paths[2], columns=["pid", "hid", "syear", 'xp13101' , 'xp13102', 'xp2701', 'xp7302','xp7202','xp28', 'xp3001' ,  'xp5701' , 'xp0102' , 'xp2702', 'xp149']).set_index(['pid', 'hid'])
df_07 = data3.rename(columns= {'xp13101':'gender','xp13102': 'year_birth' ,'xp2701': 'turnover_intention' , 'xp7302': 'wage_lastmonth','xp7202': 'overtime','xp28': 'new_job', 'xp3001': 'reason_new_job',  'xp5701' : 'commute_distance' , "xp0102" : "work_satisfaction" , 'xp2702' : "fear_losingjob" , 'xp149' : 'life_satisfaction'})

In [54]:
# read in 2007 data from work module

# adapt path and merge
hours07 = pd.read_stata(file_paths_2[2], columns=["pid","hid", "syear", "xvebzeit", "xpsbil","nace07" ,"betr07", "xerwzeit", "xbilzeit"]).set_index(['pid', 'hid'])
work07 = hours07.rename(columns={'xvebzeit': 'working_hours', "xpsbil": "school_degree", "nace07": "sector","betr07": "firmsize", "xerwzeit": "tenure" , "xbilzeit" : "years_educ"})

 ### 2. Define Functions and mappings for cleaning data

In [55]:
# mapping for reciprocity questions: same scale for all
reciprocity_questions_mapping = {
    '[1] Trifft ueberhaupt nicht zu': 1,
    '[2] Skala 1-7': 2,
    '[3] Skala 1-7': 3,
    '[4] Skala 1-7': 4,
    '[5] Skala 1-7': 5,
    '[6] Skala 1-7': 6,
    '[7] Trifft voll zu': 7,
    '[-1] keine Angabe': -1,
}
## mapping for recognition questions: binary -> binary -> unfair treatment: No -> later 1 fair treatment: Yes -> 0
recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Ja': 2,
    '[2] Nein': 1,
}
# felt recog mapping
felt_recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Gar nicht': 1,
    '[2] Maessig': 2,
    '[3] Stark': 3,
    '[4] Sehr stark': 4,
}
# mapping for firmsize -> we need to recode this in a sensible way: jumps are the same: first change: selbstständig to 0
firmsize_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Unter  5': 1,
    '[2] 5 bis 10': 2,
    '[3] 11 bis unter 20': 3,
    '[4] bis 90: unter 20': 4,
    '[5] 91-04: 5 bis unter 20': 5,
    '[6] 20 bis unter 100': 6,
    '[7] 100 bis unter 200': 7,
    '[8] bis 98: 20 bis unter 200': 8,
    '[9] 200 bis unter 2000': 9,
    '[10] 2000 und mehr': 10,
    '[11] Selbstaendig-ohne Mitarb.': 0,
}
# mapping new job into binary variable
new_job_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1, 
    '[1] Ja': 1, 
    '[2] Nein': 2,
    '[3] Ja, nach Datenpruefung': 1,
}
# mapping for job satisfaction: split up into binary with roughly equal value counts for simplicity: might change that later to categories,
satisfaction_mapping = {
    '[0] 0 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[1] 1 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[2] 2 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[3] 3 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[4] 4 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[5] 5 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 2,
    '[6] 6 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[7] 7 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[8] 8 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[9] 9 Zufrieden: Skala 0-Niedrig bis 10-Hoch': 1,
    '[10] 10 Zufrieden: Skala 0-Niedrig bis 10-Hoc': 1,
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
}
turnover_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[0] 0% wahrscheinlich': 2,
    '[10] 10% wahrscheinlich': 1,
    '[20] 20% wahrscheinlich': 1,
    '[30] 30% wahrscheinlich': 1,
    '[40] 40% wahrscheinlich': 1, 
    '[50] 50% wahrscheinlich': 1,
    '[60] 60% wahrscheinlich': 1,
    '[70] 70% wahrscheinlich': 1,
    '[80] 80% wahrscheinlich': 1,
    '[90] 90% wahrscheinlich': 1,
    '[100] 100% wahrscheinlich': 1,
}
# mapping for turnover intention robustness check: Cardinal
turnover_mapping_cardinal = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[0] 0% wahrscheinlich': 0,
    '[10] 10% wahrscheinlich': 10,
    '[20] 20% wahrscheinlich': 20,
    '[30] 30% wahrscheinlich': 30,
    '[40] 40% wahrscheinlich': 40, 
    '[50] 50% wahrscheinlich': 50,
    '[60] 60% wahrscheinlich': 60,
    '[70] 70% wahrscheinlich': 70,
    '[80] 80% wahrscheinlich': 80,
    '[90] 90% wahrscheinlich': 90,
    '[100] 100% wahrscheinlich': 100,
}
# mapping for new job to easier remove negatives
reason_new_job_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1, 
    '[1] Erstmals erwerbstaetig': 1,
    '[2] Wieder erwerbstaetig': 2,
    '[3] Stelle bei neuen Arbeitgeber': 3,  
    '[4] Uerbnommen von Betrieb': 4,
    '[5] Stellenwechsel im Betrieb': 5, 
    '[6] Selbstaendig geworden': 6,
}
# mapping for school degree: to easier remove negatives
school_degree_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[1] Hauptschulabschluss': 1,
    '[2] Realschulabschluss': 2,
    '[3] Fachhochschulreife': 3,
    '[4] Abitur': 4,
    '[5] Anderer Abschluss': 5,
    '[6] Ohne Abschluss verlassen': 6,
    '[7] Noch kein Abschluss': 7,
    '[8] Keine Schule besucht': 8,
}
# sector_map
sector_map = {
    "[1] Landwirtschaft und  Jagd": 1,
    "[2] Forstwirtschaft": 2,
    "[5] Fischerei und Fischzucht": 5,
    "[10] Kohlenbergbau, Torfgewinnung": 10,
    "[11] Gewinnung von Erdöl und Erdgas, Erbringung damit verbundener Dienstleistungen": 11,
    "[12] Bergbau auf Uran- und Thoriumerze": 12,
    "[13] Erzbergbau": 13,
    "[14] Gewinnung von Steinen und Erden, sonstiger Bergbau": 14,
    "[15] Herstellung von Nahrungs- und Futtermitteln sowie Getränken": 15,
    "[16] Tabakverarbeitung": 16,
    "[17] Herstellung von Textilien": 17,
    "[18] Herstellung von Bekleidung": 18,
    "[19] Herstellung von Leder und Lederwaren": 19,
    "[20] Herstellung von Holz sowie Holz-, Kork- und Flechtwaren (ohne Herstellung von Möbeln)": 20,
    "[21] Herstellung von Papier, Pappe und Waren daraus": 21,
    '[22] Herstellung von Verlags- und Druckerzeugnissen,  Vervielfältigung von bespielten Ton-, Bild- und Datenträgern': 22,
    "[23] Kokerei, Mineralölverarbeitung, Herstellung und Verarbeitung von Spalt- und Brutstoffen": 23,
    "[24] Herstellung von chemischen Erzeugnissen": 24,
    "[25] Herstellung von Gummi- und Kunststoffwaren": 25,
    "[26] Herstellung von Glas und Glaswaren, Keramik, Verarbeitung von Steinen und Erden": 26,
    "[27] Metallerzeugung und -bearbeitung": 27,
    "[28] Herstellung von Metallerzeugnissen": 28,
    "[29] Maschinenbau": 29,
    "[31] Herstellung von Geräten der Elektrizitätserzeugung, -verteilung u. Ä.": 31,
    "[30] Herstellung von Büromaschinen, Datenverarbeitungsgeräten und -einrichtungen": 30,
    "[32] Rundfunk- und Nachrichtentechnik": 32,
    "[33] Medizin-, Mess-, Steuer- und Regelungstechnik, Optik, Herstellung von Uhren": 33,
    "[34] Herstellung von Kraftwagen und Kraftwagenteilen": 34,
    "[35] Sonstiger Fahrzeugbau": 35,
    "[36] Herstellung von Möbeln, Schmuck, Musikinstrumenten, Sportgeräten, Spielwaren und sonstigen Erzeugnissen": 36,
    "[37] Rückgewinnung": 37,
    "[40] Energieversorgung": 40,
    "[41] Wasserversorgung": 41,
    "[45] Bau": 45,
    "[50] Kraftfahrzeughandel; Instandhaltung und Reparatur von Kraftfahrzeugen; Tankstellen": 50,
    "[51] Handelsvermittlung und Großhandel (ohne Handel mit Kraftfahrzeugen)": 51,
    "[52] Einzelhandel (ohne Handel mit Kraftfahrzeugen und ohne Tankstellen); Reparatur von Gebrauchsgütern": 52,
    "[55] Beherbergungs- und Gaststätten": 55,
    "[60] Landverkehr; Transport in Rohrfernleitungen": 60,
    "[61] Schifffahrt": 61,
    "[62] Luftfahrt": 62,
    "[63] Hilfs- und Nebentätigkeiten für den Verkehr; Verkehrsvermittlung": 63,
    "[64] Nachrichtenübermittlung": 64,
    "[65] Kreditinstitute": 65,
    "[66] Versicherungen (ohne Sozialversicherung)": 66,
    "[67] Mit den Kreditinstituten und Versicherungen verbundene Tätigkeiten": 67,
    "[70] Grundstücks- und Wohnungswesen": 70,
    "[71] Vermietung beweglicher Sachen ohne Bedienungspersonal": 71,
    "[72] Datenverarbeitung und Datenbanken": 72,
    "[73] Forschung und Entwicklung": 73,
    "[74] Erbringung von unternehmensbezogenen Dienstleistungen": 74,
    "[75] Öffentliche Verwaltung, Verteidigung, Sozialversicherung": 75,
    "[80] Erziehung und Unterricht": 80,
    "[85] Gesundheits-, Veterinär- und Sozialwesen": 85,
    "[90] Abwasser- und Abfallbeseitigung und sonstige Entsorgung": 90,
    "[91] Interessenvertretungen sowie kirchliche und sonstige Vereinigungen (ohne Sozialwesen, Kultur und Sport)": 91,
    "[92] Kultur, Sport und Unterhaltung": 92,
    "[93] Erbringung von sonstigen Dienstleistungen": 93,
    "[95] Private Haushalte mit Hauspersonal": 95,					
    "[96] Industrie - ohne weitere Zuordnung": 96,					
    "[97] Handwerk - ohne weitere Zuordnung": 97,					
    "[98] Dienstleistungen ohne weitere Zuordnung": 98,					
    "[99] Exterritoriale Organisationen und Körperschaften": 99,				
    "[100] Produzierendes Gewerbe ohne w.Zuordnung": 100,
    "[-1] keine Angabe": 3,
    '[-2] trifft nicht zu': 0, 
    "[-3] unplausibler Wert": -3,
    "[-4] unzulaessige Mehrfachantwort": -4, 
    "[-5] in Fragebogenversion nicht enthalten": -5,
    "[-6] Fragebogenversion mit geaenderter Filterfuehrung": -6, 
    "[-7] nur in weniger eingeschraenkter Edition verfuegbar": -7,
    "[-8] Frage in diesem Jahr nicht Teil des Frageprogramms": -8,
}
# reversed mapping to redo changes
reversed_mapping_reason = {v: k for k, v in reason_new_job_mapping.items()}
reversed_mapping_schoold = {v: k for k, v in school_degree_mapping.items()}
reversed_mapping_sector = {v: k for k, v in sector_map.items()}

## function for recoding values and dropping missing

def recode_categoricals(inputdf,rc_cardinal = 0):
    """
        Applies recoding to categoricals to easier drop n.a.ns

        Input:
            - inputdf : merged dataframe
            - rc_cardinal: optional argument: if == 1 turnover intentions will be coded as cardinal instead of binary 
            
        Output:
            - df : cleaned Dataframe

    """
    merged = inputdf  
    
    # recode Gender variable
    merged['gender'].replace('[2] Weiblich', 2,inplace=True)
    merged['gender'].replace('[1] Maennlich', 1,inplace=True)
    # recode reciprocity variables
    merged[["similar_problems","take_revenge","insult_back"]] = merged[["similar_problems","take_revenge","insult_back"]].apply(lambda x: x.map(reciprocity_questions_mapping))
    # recode recognition variables
    merged[["recog_sup","recog_effort","recog_personal","recog_pay","jobatrisk"]] = merged[["recog_sup","recog_effort","recog_personal","recog_pay","jobatrisk"]].apply(lambda x: x.map(recog_mapping))
    # recode felt recognition variables
    merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]] = merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]].apply(lambda x: x.map(felt_recog_mapping))
    # recode firm size
    merged['firmsize'] = merged['firmsize'].map(firmsize_mapping)
    # recode new job reason variable
    merged['reason_new_job'] = merged['reason_new_job'].map(reason_new_job_mapping)
    # recode job change variable
    merged['new_job']= merged['new_job'].map(new_job_mapping)
    # recode sector
    merged['sector'] = merged['sector'].map(sector_map)
    # recode job satisfaction
    merged['work_satisfaction']= merged['work_satisfaction'].map(satisfaction_mapping)
    merged['life_satisfaction'] = merged['life_satisfaction'].map(satisfaction_mapping)
    # recode turnover intention variable
    merged['fear_losingjob'] = merged['fear_losingjob'].map(turnover_mapping_cardinal)
    if rc_cardinal == 1:
        merged['turnover_intention'] = merged['turnover_intention'].map(turnover_mapping_cardinal)
    else:
        merged['turnover_intention'] = merged['turnover_intention'].map(turnover_mapping)
    
    # recode school degree
    merged['school_degree'] = merged['school_degree'].map(school_degree_mapping)
    
    output = merged
    return output

# Merge dataframes: a bit tough to read as its nested, merges 4 dataframes: 2005,2006,2007 and 2007gen

def merge_and_clean(df_05,df_06,df_07,work07,rc_cardinal = 0,rc_rec_binary=0):
    """
    merges data from different years, applies recoding to categoricals and constructs additional variables.

    Input:
        - df_05 : Pd.Dataframe contains reciprocity measures
        - df_06 : Pd.Dataframe contains unfair treatment measures
        - df_07 : Pd.Dataframe contains outcome and controls
        - work07 : Pd.Dataframe contains additional controls

    Output:
        - df : cleaned Dataframe

    """
    allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
    recoded = recode_categoricals(allmerged_df,rc_cardinal).astype('int')
    # replaces negative values with n.a.n 
    recoded = recoded.mask(recoded < 0, np.nan)
    # recode sector back intro category
    
    recoded["wage_lastmonth"] = np.log(recoded["wage_lastmonth"])
    recoded = recoded[recoded["wage_lastmonth"] != -np.inf] 
    # construct avg reciprocity measure
    recoded['avg_rec'] = recoded[['take_revenge', 'similar_problems', 'insult_back']].mean(axis=1)
    # For robustness check: option to construct binary reciprocity measure.
    if rc_rec_binary == 1:
        recoded['binary_rec'] = recoded['avg_rec'].apply(binary_reciprocity)
    
    # construct age, potential experience and age^2
    recoded['age'] = 2007 - recoded['year_birth']
    recoded["potential_experience"] = pow((recoded["age"] - 18), 2)
    recoded["age_squared"] = (recoded["age"] ** 2) / 100
    recoded["tenure_squared"] = (recoded["tenure"] ** 2) / 100
    # recode categoricals back to make it better readable
    recoded["reason_new_job"] = recoded["reason_new_job"].map(reversed_mapping_reason)
    recoded["school_degree"] = recoded["school_degree"].map(reversed_mapping_schoold)
    recoded["sector"] = recoded["sector"].map(reversed_mapping_sector)
    # transform binary variables with 1 and 2 into 1 and 0
    columns_to_transform = ["recog_sup","recog_effort", "recog_pay", "recog_personal" ,"gender", "turnover_intention", "new_job" , "jobatrisk"]

    # Iterate over the columns and replace the values 2 with 0 
    for col in columns_to_transform:
        recoded[col] = recoded[col].replace({2: 0})

    # save df somewhere so its not muted when repeatedly executing this cell: Can later transform that into functions
    df = recoded
    
    return df

# Add Mincer Wage Regression and adds its residuals to the Dataframe

def add_mincer_residuals(cleaneddata):
    """
    Performs a Minzer-wage Regression on the cleaned dataset and adds
    the residuals to the dataframe. This will be used as the wage controls

        Input: 
            - cleaneddata(pd.DataFrame) : merged and recoded dataset

        Output: 
            - df_cleaned(pd.DataFrame)  : samedataset with Minzer-residuals
    
    """



    df_cleaned = cleaneddata
    # specify which columns to drop from our dataframe
    df_mincer = cleaneddata.drop(columns=['syear_x', 'similar_problems', 'take_revenge', 'insult_back','syear_y', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'syear_y', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_sup', 'age', 'commute_distance', 'recog_effort', 'working_hours', 'turnover_intention', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction' , 'sector' , 'tenure_squared' , 'fear_losingjob' , 'jobatrisk' , 'life_satisfaction'])
    # Convert 'gender' and 'sector' columns to categorical data type
    for col in ['gender']:
        df_mincer[col] = df_mincer[col].astype('category')
    df_mincer = df_mincer.dropna()
    # Define the dependent variable
    y = df_mincer['wage_lastmonth']
    # Define the independent variables
    X = df_mincer[['gender', 'firmsize', 'tenure', 'years_educ', 'potential_experience', 'age_squared']]

    # Add a constant term to the independent variables
    X = sm.add_constant(X)

    # Fit the Mincer wage regression model
    mincer_model = sm.OLS(y, X).fit()
    
    # Create a new column in the dataframe with the same name as the residuals array/ delete relative wage entries
    df_cleaned['mincer_residuals'] = None
    # Match the rows of the dataframe with the values in the residuals array using the index
    df_cleaned.loc[df_cleaned.index, 'mincer_residuals'] = mincer_model.resid

    return df_cleaned

### Inlcude People who switched their jobs in 2006-2007 with 2006 controls

In [56]:
def include_jobchangers(data07,onlynewemployer=True,worksatisfaction=False,rc_cardinal = False):
    """
    Replaces controls of subjects which switched their jobs-from 2006-2007 with 
    controls from their job at the time.
    
    Input: 
        - data07(Pd.Dataframe)  : finished 2007 dataframe
        - onlynewemployer (optional argument) : If True only includes those who switched jobs to a new employer
        - worksatisfaction (optional argument): If True drops observations where job satisfaction between 2006 and 2007 changed 

    Output:
        - dfconcat: Dataframe where controls for people who switched their jobs are from 2006
    
    """
    df = data07
    # read in cleaned 2006 dataset:
    #  change path here : Maxie: '/Users/maxieschulze/Documents/Dokumente - MacBook Pro von Maxie/5. Semester/Research Module/ResearchModule/src/data_management/2006jobchange.csv'
    #                     Max 'C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/finalproj/src/data_management/2006jobchange.csv'  
    observations_2006 = pd.read_csv('C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/finalproj/src/data_management/2006jobchange.csv')
    # some initial datamanagement
    observations_2006.reset_index(inplace=True)
    observations_2006.set_index(["pid","hid"], inplace=True)
    observations_2006.drop(columns=observations_2006.filter(regex='^syear').columns, inplace=True)
    observations_2006.drop("index",axis=1 ,inplace =True)
    # If cardinal turnover intentions are coded to 100
    if rc_cardinal == True:
        observations_2006["turnover_intention"] = 100
    
    # drop s year columns from both dataframes
    df.drop(columns=df.filter(regex='^syear').columns, inplace=True)
    
    # optional worksatisfaction robustness check
    if worksatisfaction == True:
        satisfaction = pd.DataFrame(df['work_satisfaction']).join(pd.DataFrame(observations_2006['work_satisfaction']), on = ["pid", "hid"], lsuffix = "_07" , rsuffix ="_06")
        # list of IDs where job satisfaction changed over the past year
        ID_keep_satis = satisfaction[satisfaction["work_satisfaction_07"] == satisfaction["work_satisfaction_06"]].index
        # drops all persons from 2007 dataframe where job satisfaction changed
        df.drop(df[~df.index.isin(ID_keep_satis)].index, inplace=True)
    
    # create list of IDs of people who switched to a new employer in the last year in 2007
    if onlynewemployer == True:
        IDs_tokeep = df[(df["new_job"] == 1) & (df["reason_new_job"] == '[3] Stelle bei neuen Arbeitgeber')].index
    # if True drops people whose work satisfaction changed between 2006 and 2007
    else:
        IDs_tokeep = df[(df["new_job"] == 1)].index
    
    # drop all who changed their job in 2007 dataframe and replace 2007 controls with 2006 controls
    df.drop(df[df["new_job"] == 1].index, inplace = True)
    observations_2006.drop(observations_2006[~observations_2006.index.isin(IDs_tokeep)].index, inplace=True)
    #concat both dataframes
    dfconcat = pd.concat([df,observations_2006])
    
    return dfconcat

### 3. Analysis


In [58]:
df_OLS = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07)))
# add interaction term
df_OLS["recXrecog_sup"] = df_OLS["recog_sup"] * df_OLS["avg_rec"]

## Drop n.a.n.s
df_OLS.drop(columns=['similar_problems', 'take_revenge', 'insult_back', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_effort','wage_lastmonth', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction'], inplace=True)
df_OLS.dropna(inplace=True)
# drop missing data in regression dataframe


# Specify model
formula_main = 'turnover_intention ~ recog_sup + avg_rec + recXrecog_sup + working_hours + firmsize + tenure + tenure_squared + age + age_squared + years_educ + commute_distance + potential_experience + mincer_residuals + jobatrisk '

# Fit the regression and cluster on the sector variable
reg = smf.ols(formula_main, data=df_OLS).fit(cov_type='cluster', cov_kwds={'groups': df_OLS['sector']})

# Print the regression results
summary = reg.summary()
#print(summary.as_latex())
summary

C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\3718291701.py:268: FutureWarning: Passing 'suffixes' which cause duplicate columns {'syear_x'} in the result is deprecated and will raise a MergeError in a future version.
  allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\3718291701.py:220: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['gender'].replace('[2] Weiblich', 2,inplace=True)
C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\3718291701.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.195
Model:                            OLS   Adj. R-squared:                  0.193
Method:                 Least Squares   F-statistic:                     701.0
Date:                Thu, 19 Jan 2023   Prob (F-statistic):           2.28e-58
Time:                        14:00:47   Log-Likelihood:                -3313.6
No. Observations:                5372   AIC:                             6655.
Df Residuals:                    5358   BIC:                             6747.
Df Model:                          13                                         
Covariance Type:              cluster                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.0461      0.009      5.373      0.000       0.029       0.063
recog_sup                0.1238      0.031      3.989      0.000       0.063       0.185
avg_rec                  0.0225      0.006      3.916      0.000       0.011       0.034
recXrecog_sup           -0.0107      0.008     -1.324      0.185      -0.027       0.005
working_hours            0.0020      0.001      1.748      0.081      -0.000       0.004
firmsize                -0.0118      0.004     -3.108      0.002      -0.019      -0.004
tenure                  -0.0159      0.002     -7.948      0.000      -0.020      -0.012
tenure_squared           0.0204      0.004      4.717      0.000       0.012       0.029
age                      0.0517      0.007      7.149      0.000       0.037       0.066
age_squared             -0.1309      0.025     -5.164      0.000      -0.181      -0.081
years_educ               0.0214      0.007      3.010      0.003       0.007       0.035
commute_distance         0.0004      0.000      3.469      0.001       0.000       0.001
potential_experience     0.0010      0.000      3.284      0.001       0.000       0.002
mincer_residuals        -0.0698      0.020     -3.466      0.001      -0.109      -0.030
jobatrisk               -0.1448      0.019     -7.614      0.000      -0.182      -0.108
==============================================================================
Omnibus:                    16933.044   Durbin-Watson:                   1.850
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              377.474
Skew:                          -0.031   Prob(JB):                     1.08e-82
Kurtosis:                       1.703   Cond. No.                     6.27e+17
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The smallest eigenvalue is 1.24e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### Robustness Checks


#### 1. Recode turnover variable into cardinal variable: Optional argument in recode_categoricals

In [28]:
# optional argument ==1
df_card = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07,rc_cardinal=1)),rc_cardinal=True)
## drop n.a.ns
df_card.drop(columns=['similar_problems', 'take_revenge', 'insult_back','felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_effort','wage_lastmonth','rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' ,'work_satisfaction'], inplace = True)
# add interaction term
df_card["recXrecog_sup"] = df_card["recog_sup"] * df_card["avg_rec"]
# drop missing data in regression dataframe
df_card.dropna(inplace=True)

import statsmodels.formula.api as smf

# Specify model
formula_main = 'turnover_intention ~ recog_sup + avg_rec + recXrecog_sup + working_hours + firmsize + tenure + tenure_squared + age + age_squared + years_educ + commute_distance + potential_experience + mincer_residuals + jobatrisk'

# Fit the regression and cluster on the sector variable
reg_cardinal = smf.ols(formula_main, data=df_card).fit(cov_type='cluster', cov_kwds={'groups': df_card['sector']})

# Print the regression results
summary = reg_cardinal.summary()
#print(summary.as_latex())
summary

C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:267: FutureWarning: Passing 'suffixes' which cause duplicate columns {'syear_x'} in the result is deprecated and will raise a MergeError in a future version.
  allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:220: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['gender'].replace('[2] Weiblich', 2,inplace=True)
C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.224
Model:                            OLS   Adj. R-squared:                  0.222
Method:                 Least Squares   F-statistic:                     256.0
Date:                Tue, 17 Jan 2023   Prob (F-statistic):           4.76e-46
Time:                        23:33:51   Log-Likelihood:                -25021.
No. Observations:                5380   AIC:                         5.007e+04
Df Residuals:                    5366   BIC:                         5.016e+04
Df Model:                          13                                         
Covariance Type:              cluster                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                5.7259      0.573      9.997      0.000       4.603       6.848
recog_sup                8.3798      1.429      5.862      0.000       5.578      11.181
avg_rec                  0.8030      0.225      3.561      0.000       0.361       1.245
recXrecog_sup           -0.4742      0.400     -1.186      0.235      -1.258       0.309
working_hours            0.0718      0.055      1.305      0.192      -0.036       0.180
firmsize                -0.3755      0.152     -2.476      0.013      -0.673      -0.078
tenure                  -1.5030      0.129    -11.611      0.000      -1.757      -1.249
tenure_squared           2.6795      0.281      9.525      0.000       2.128       3.231
age                      5.1298      0.475     10.804      0.000       4.199       6.060
age_squared            -16.7033      1.687     -9.900      0.000     -20.010     -13.396
years_educ               1.0564      0.219      4.815      0.000       0.626       1.486
commute_distance         0.0283      0.008      3.612      0.000       0.013       0.044
potential_experience     0.1698      0.019      8.770      0.000       0.132       0.208
mincer_residuals        -6.1042      1.074     -5.686      0.000      -8.208      -4.000
jobatrisk              -10.1536      1.144     -8.872      0.000     -12.397      -7.911
==============================================================================
Omnibus:                      840.036   Durbin-Watson:                   1.926
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1320.943
Skew:                           1.087   Prob(JB):                    1.45e-287
Kurtosis:                       4.080   Cond. No.                     4.12e+17
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The smallest eigenvalue is 2.87e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### 2. Creating dummies and include 7 interaction terms : Decide for one of the 3 questions for simplicity

When treating the categories as numerical: we are making assumptions about the differences between the scale items. If those distances can be considered equal at all levels, then it is reasonable to treat reciprocity as numerical. (i.e a one unit change from 1 to 2 is equivalent to a one unit change from 6 to 7)


For dummy coding we need to exclude one of the categories in the dataframe and make it the reference category: This will be the lowest level of reciprocity 1 and will be coded as zero.  so rec2 rec3 rec4 , ... and their interaction terms with unfair treatment stay in the regression.

rec2 is then interpreted as the mean of turnover intentions in the rec2 group - the mean of turnover intentions in the rec1 group (reference group) holding everything else constant

In [43]:
# 1. optional argument:  rc_binary ==1
dfrc_dummy = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07)))
# specify columns which we need for regression by dropping everything else
dfrc_dummy.drop(columns=['similar_problems', 'take_revenge', 'insult_back','felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay','year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_effort','wage_lastmonth', 'avg_rec', 'work_satisfaction'], inplace = True)
# add interaction terms
for col in ['rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7']:
    dfrc_dummy = dfrc_dummy.assign(**{col + '_X_recog_sup': dfrc_dummy[col] * dfrc_dummy['recog_sup']})
# drop missing data in regression dataframe
dfrc_dummy = dfrc_dummy.dropna()


# Specify model
formula_main = 'turnover_intention ~ recog_sup  + rec2 + rec3 + rec4 + rec5 + rec6 + rec7 + rec2_X_recog_sup + rec3_X_recog_sup + rec4_X_recog_sup + rec5_X_recog_sup + rec6_X_recog_sup + rec7_X_recog_sup+ working_hours + firmsize + tenure + tenure_squared + age + age_squared + years_educ + commute_distance + potential_experience + mincer_residuals + jobatrisk'

# Fit the regression and cluster on the sector variable
reg = smf.ols(formula_main, data=dfrc_dummy).fit(cov_type='cluster', cov_kwds={'groups': dfrc_dummy['sector']})

# Print the regression results
summary = reg.summary()
#print(summary.as_latex())
summary

C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:267: FutureWarning: Passing 'suffixes' which cause duplicate columns {'syear_x'} in the result is deprecated and will raise a MergeError in a future version.
  allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:220: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['gender'].replace('[2] Weiblich', 2,inplace=True)
C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.205
Method:                 Least Squares   F-statistic:                     591.1
Date:                Wed, 18 Jan 2023   Prob (F-statistic):           8.74e-60
Time:                        16:41:53   Log-Likelihood:                -3275.5
No. Observations:                5380   AIC:                             6599.
Df Residuals:                    5356   BIC:                             6757.
Df Model:                          23                                         
Covariance Type:              cluster                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.0449      0.008      5.839      0.000       0.030       0.060
recog_sup                0.1172      0.026      4.453      0.000       0.066       0.169
rec2                     0.1642      0.015     11.236      0.000       0.136       0.193
rec3                     0.1535      0.020      7.848      0.000       0.115       0.192
rec4                     0.1008      0.023      4.396      0.000       0.056       0.146
rec5                     0.1600      0.027      5.898      0.000       0.107       0.213
rec6                     0.1982      0.046      4.272      0.000       0.107       0.289
rec7                     0.0515      0.044      1.167      0.243      -0.035       0.138
rec2_X_recog_sup        -0.0661      0.032     -2.078      0.038      -0.129      -0.004
rec3_X_recog_sup         0.0047      0.039      0.121      0.904      -0.071       0.080
rec4_X_recog_sup        -0.0175      0.032     -0.544      0.587      -0.081       0.046
rec5_X_recog_sup        -0.0580      0.045     -1.279      0.201      -0.147       0.031
rec6_X_recog_sup        -0.0778      0.069     -1.122      0.262      -0.214       0.058
rec7_X_recog_sup        -0.0458      0.066     -0.696      0.487      -0.175       0.083
working_hours            0.0018      0.001      1.553      0.120      -0.000       0.004
firmsize                -0.0120      0.004     -3.266      0.001      -0.019      -0.005
tenure                  -0.0159      0.002     -7.862      0.000      -0.020      -0.012
tenure_squared           0.0198      0.004      4.616      0.000       0.011       0.028
age                      0.0504      0.006      7.866      0.000       0.038       0.063
age_squared             -0.1273      0.023     -5.600      0.000      -0.172      -0.083
years_educ               0.0196      0.007      2.826      0.005       0.006       0.033
commute_distance         0.0004      0.000      3.852      0.000       0.000       0.001
potential_experience     0.0009      0.000      3.512      0.000       0.000       0.001
mincer_residuals        -0.0712      0.019     -3.654      0.000      -0.109      -0.033
jobatrisk               -0.1489      0.018     -8.110      0.000      -0.185      -0.113
==============================================================================
Omnibus:                     9280.478   Durbin-Watson:                   1.849
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              359.116
Skew:                          -0.040   Prob(JB):                     1.04e-78
Kurtosis:                       1.737   Cond. No.                     2.13e+17
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The smallest eigenvalue is 1.08e-25. This

##### Robustness Check : Drop people whose job satisfaction level changed

In [31]:
dfworksatisfation = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07)),worksatisfaction = True)
# specify columns which we need for regression by dropping everything else

df_rc_ws = dfworksatisfation.drop(columns=['similar_problems', 'take_revenge', 'insult_back', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_effort','wage_lastmonth', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' ,'work_satisfaction'])
#  included 'recog_effort'again , excluded overtime due to sample size and wage last month """
# add interaction term
df_rc_ws["recXrecog_sup"] = df_rc_ws["recog_sup"] * df_rc_ws["avg_rec"]
# drop missing data in regression dataframe
df_rc_ws.dropna(inplace=True)

formula_main = 'turnover_intention ~ recog_sup + avg_rec + recXrecog_sup + working_hours + firmsize + tenure + tenure_squared + age + age_squared + years_educ + commute_distance + potential_experience + mincer_residuals + jobatrisk'
# Fit the regression and cluster on the sector variable
reg = smf.ols(formula_main, data=df_rc_ws).fit(cov_type='cluster', cov_kwds={'groups': df_rc_ws['sector']})

# Print the regression results
summary = reg.summary()
#print(summary.as_latex())
summary

C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:267: FutureWarning: Passing 'suffixes' which cause duplicate columns {'syear_x'} in the result is deprecated and will raise a MergeError in a future version.
  allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:220: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['gender'].replace('[2] Weiblich', 2,inplace=True)
C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.186
Model:                            OLS   Adj. R-squared:                  0.183
Method:                 Least Squares   F-statistic:                     389.3
Date:                Tue, 17 Jan 2023   Prob (F-statistic):           2.03e-50
Time:                        23:34:31   Log-Likelihood:                -2691.4
No. Observations:                4331   AIC:                             5411.
Df Residuals:                    4317   BIC:                             5500.
Df Model:                          13                                         
Covariance Type:              cluster                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.0486      0.008      5.928      0.000       0.033       0.065
recog_sup                0.1427      0.031      4.625      0.000       0.082       0.203
avg_rec                  0.0211      0.006      3.422      0.001       0.009       0.033
recXrecog_sup           -0.0088      0.008     -1.082      0.279      -0.025       0.007
working_hours            0.0017      0.001      1.582      0.114      -0.000       0.004
firmsize                -0.0115      0.004     -2.964      0.003      -0.019      -0.004
tenure                  -0.0151      0.002     -6.901      0.000      -0.019      -0.011
tenure_squared           0.0200      0.005      4.099      0.000       0.010       0.030
age                      0.0523      0.007      7.385      0.000       0.038       0.066
age_squared             -0.1387      0.024     -5.734      0.000      -0.186      -0.091
years_educ               0.0223      0.007      2.999      0.003       0.008       0.037
commute_distance         0.0004      0.000      2.842      0.004       0.000       0.001
potential_experience     0.0011      0.000      3.916      0.000       0.001       0.002
mincer_residuals        -0.0552      0.022     -2.515      0.012      -0.098      -0.012
jobatrisk               -0.1502      0.022     -6.806      0.000      -0.193      -0.107
==============================================================================
Omnibus:                    36073.693   Durbin-Watson:                   1.854
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              318.400
Skew:                           0.036   Prob(JB):                     7.25e-70
Kurtosis:                       1.674   Cond. No.                     5.87e+17
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The smallest eigenvalue is 1.12e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### Robustness Check 

Avg reciprocity measure over years



In [32]:
df_OLS_avg = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07)))
# specify columns which we need for regression by dropping everything else
df_OLS_avg.drop(columns=['similar_problems', 'take_revenge', 'insult_back', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_effort','wage_lastmonth', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction','avg_rec'], inplace = True)
# Load avg reciprocity measures over the years


# Max 'C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/finalproj/src/data_management/rec_avgyears.csv'
# Maxie '/Users/maxieschulze/Documents/Dokumente - MacBook Pro von Maxie/5. Semester/Research Module/ResearchModule/src/data_management/rec_avgyears.csv'
avg_reciprocity = pd.read_csv('C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/finalproj/src/data_management/rec_avgyears.csv')
avg_reciprocity.reset_index(inplace=True)
avg_reciprocity.set_index(["pid","hid"], inplace=True)
avg_reciprocity.drop("index",axis=1 ,inplace =True)

df_avg_years = pd.merge(df_OLS_avg,avg_reciprocity, on=["pid","hid"])

# add interaction term
df_avg_years["recXrecog_sup"] = df_avg_years["recog_sup"] * df_avg_years["avg_rec"]
# drop missing data in regression dataframe
df_avg_years.dropna(inplace=True)

formula_main = 'turnover_intention ~ recog_sup + avg_rec + recXrecog_sup + working_hours + firmsize + tenure + tenure_squared + age + age_squared + years_educ + commute_distance + potential_experience + mincer_residuals + jobatrisk'
# Fit the regression and cluster on the sector variable
reg = smf.ols(formula_main, data=df_avg_years).fit(cov_type='cluster', cov_kwds={'groups': df_avg_years['sector']})

# Print the regression results
summary = reg.summary()
#print(summary.as_latex())
summary

C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:267: FutureWarning: Passing 'suffixes' which cause duplicate columns {'syear_x'} in the result is deprecated and will raise a MergeError in a future version.
  allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:220: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['gender'].replace('[2] Weiblich', 2,inplace=True)
C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.196
Model:                            OLS   Adj. R-squared:                  0.194
Method:                 Least Squares   F-statistic:                     610.6
Date:                Tue, 17 Jan 2023   Prob (F-statistic):           1.13e-56
Time:                        23:34:35   Log-Likelihood:                -3317.5
No. Observations:                5380   AIC:                             6663.
Df Residuals:                    5366   BIC:                             6755.
Df Model:                          13                                         
Covariance Type:              cluster                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.0455      0.008      5.443      0.000       0.029       0.062
recog_sup                0.1090      0.035      3.103      0.002       0.040       0.178
avg_rec                  0.0245      0.006      4.056      0.000       0.013       0.036
recXrecog_sup           -0.0062      0.010     -0.638      0.524      -0.025       0.013
working_hours            0.0019      0.001      1.682      0.093      -0.000       0.004
firmsize                -0.0116      0.004     -3.059      0.002      -0.019      -0.004
tenure                  -0.0161      0.002     -8.043      0.000      -0.020      -0.012
tenure_squared           0.0208      0.004      4.806      0.000       0.012       0.029
age                      0.0509      0.007      7.033      0.000       0.037       0.065
age_squared             -0.1290      0.025     -5.245      0.000      -0.177      -0.081
years_educ               0.0220      0.007      3.078      0.002       0.008       0.036
commute_distance         0.0004      0.000      3.518      0.000       0.000       0.001
potential_experience     0.0010      0.000      3.347      0.001       0.000       0.002
mincer_residuals        -0.0694      0.020     -3.499      0.000      -0.108      -0.031
jobatrisk               -0.1450      0.019     -7.639      0.000      -0.182      -0.108
==============================================================================
Omnibus:                    16326.953   Durbin-Watson:                   1.846
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              376.907
Skew:                          -0.029   Prob(JB):                     1.43e-82
Kurtosis:                       1.705   Cond. No.                     4.38e+17
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The smallest eigenvalue is 2.54e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

Also does not change much

##### Realized Turnover in 2007 -> turnover intentions = 1 else 0 with 2006 controls

In [33]:
def realizedturnover(df_07):
    # gets people who realized turnover in 2006-2007
    jobchangers = df_07[df_07["new_job"] == '[1] Ja'].index

    observations_2006 = pd.read_csv('C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/finalproj/src/data_management/2006jobchange.csv')
    # Max 'C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/finalproj/src/data_management/2006jobchange.csv'
    # Maxie '/Users/maxieschulze/Documents/Dokumente - MacBook Pro von Maxie/5. Semester/Research Module/ResearchModule/src/data_management/2006jobchange.csv'
    # some initial datamanagement
    observations_2006.reset_index(inplace=True)
    observations_2006.set_index(["pid","hid"], inplace=True)
    observations_2006.drop(columns=observations_2006.filter(regex='^syear').columns, inplace=True)
    observations_2006.drop("index",axis=1 ,inplace =True)
    # define people who changed their jobs
    jobstayers = observations_2006[~observations_2006.index.isin(jobchangers)]
    jobstayers["turnover_intention"] = 0
    # drop all to concatenate both: not elegant all which did not change job
    observations_2006.drop(observations_2006[~observations_2006.index.isin(jobchangers)].index, inplace=True)

    subset_2006 = pd.concat([jobstayers,observations_2006])

    return subset_2006

In [35]:
df_real = realizedturnover(df_07)
df_real.drop(columns=['similar_problems', 'take_revenge', 'insult_back', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_effort','wage_lastmonth', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction'], inplace = True)
#  included 'recog_effort'again , excluded overtime due to sample size and wage last month """
# add interaction term
df_real["recXrecog_sup"] = df_real["recog_sup"] * df_real["avg_rec"]
# drop missing data in regression dataframe
df_real.dropna(inplace=True)

formula_main = 'turnover_intention ~ recog_sup + avg_rec + recXrecog_sup + working_hours + firmsize + tenure + tenure_squared + age + age_squared + years_educ + commute_distance + potential_experience + mincer_residuals + jobatrisk'
# Fit the regression and cluster on the sector variable
reg = smf.ols(formula_main, data=df_real).fit(cov_type='cluster', cov_kwds={'groups': df_real['sector']})

# Print the regression results
reg.summary()

C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\3458904763.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobstayers["turnover_intention"] = 0


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.076
Model:                            OLS   Adj. R-squared:                  0.074
Method:                 Least Squares   F-statistic:                     130.9
Date:                Tue, 17 Jan 2023   Prob (F-statistic):           5.73e-38
Time:                        23:34:53   Log-Likelihood:                -250.42
No. Observations:                6581   AIC:                             528.8
Df Residuals:                    6567   BIC:                             623.9
Df Model:                          13                                         
Covariance Type:              cluster                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.0199      0.005      3.878      0.000       0.010       0.030
recog_sup                0.0041      0.012      0.326      0.745      -0.020       0.028
avg_rec                  0.0007      0.003      0.236      0.814      -0.005       0.007
recXrecog_sup        -4.928e-05      0.004     -0.013      0.990      -0.007       0.007
working_hours           -0.0005      0.000     -1.233      0.218      -0.001       0.000
firmsize                -0.0029      0.001     -2.003      0.045      -0.006   -6.25e-05
tenure                  -0.0146      0.002     -9.660      0.000      -0.018      -0.012
tenure_squared           0.0346      0.004      9.383      0.000       0.027       0.042
age                      0.0199      0.004      5.098      0.000       0.012       0.028
age_squared             -0.0572      0.015     -3.761      0.000      -0.087      -0.027
years_educ               0.0057      0.001      5.738      0.000       0.004       0.008
commute_distance         0.0002   9.72e-05      1.822      0.068   -1.34e-05       0.000
potential_experience     0.0005      0.000      3.021      0.003       0.000       0.001
mincer_residuals        -0.0151      0.008     -1.877      0.060      -0.031       0.001
jobatrisk               -0.0453      0.009     -5.158      0.000      -0.063      -0.028
==============================================================================
Omnibus:                     3817.012   Durbin-Watson:                   0.145
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            23895.524
Skew:                           2.882   Prob(JB):                         0.00
Kurtosis:                      10.343   Cond. No.                     1.03e+18
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The smallest eigenvalue is 5.5e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

### Other measure for unfair treatment : take additional information into account how much they are bothered by it

In [37]:
df_OLS = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07)))
# add interaction term
idstostayat1 = df_OLS[ (df_OLS["felt_recog_sup"] == 3.0) | (df_OLS["felt_recog_sup"] == 4.0) | (df_OLS["felt_recog_sup"] == 2.0)].index
df_OLS['recog_sup'][~df_OLS.index.isin(idstostayat1)] = 0
df_OLS["recXrecog_sup"] = df_OLS["recog_sup"] * df_OLS["avg_rec"]
## Drop n.a.n.s
df_OLS.drop(columns=['similar_problems', 'take_revenge', 'insult_back', 'felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_effort','wage_lastmonth', 'rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' , 'work_satisfaction'], inplace=True)
df_OLS.dropna(inplace=True)
## change all which are unbothered to unfair treatment 0

 
## add interaction term and reestimate





# Specify model
formula_main = 'turnover_intention ~ recog_sup + avg_rec + recXrecog_sup + working_hours + firmsize + tenure + tenure_squared + age + age_squared + years_educ + commute_distance + potential_experience + mincer_residuals + jobatrisk'

# Fit the regression and cluster on the sector variable
reg = smf.ols(formula_main, data=df_OLS).fit(cov_type='cluster', cov_kwds={'groups': df_OLS['sector']})

# Print the regression results
summary = reg.summary()
#print(summary.as_latex())
summary


C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:267: FutureWarning: Passing 'suffixes' which cause duplicate columns {'syear_x'} in the result is deprecated and will raise a MergeError in a future version.
  allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:220: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['gender'].replace('[2] Weiblich', 2,inplace=True)
C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.196
Model:                            OLS   Adj. R-squared:                  0.194
Method:                 Least Squares   F-statistic:                     618.5
Date:                Tue, 17 Jan 2023   Prob (F-statistic):           7.90e-57
Time:                        23:35:40   Log-Likelihood:                -3329.7
No. Observations:                5402   AIC:                             6687.
Df Residuals:                    5388   BIC:                             6780.
Df Model:                          13                                         
Covariance Type:              cluster                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.0466      0.009      5.364      0.000       0.030       0.064
recog_sup                0.1421      0.035      4.086      0.000       0.074       0.210
avg_rec                  0.0219      0.006      3.833      0.000       0.011       0.033
recXrecog_sup           -0.0125      0.010     -1.309      0.191      -0.031       0.006
working_hours            0.0020      0.001      1.713      0.087      -0.000       0.004
firmsize                -0.0115      0.004     -3.015      0.003      -0.019      -0.004
tenure                  -0.0157      0.002     -8.042      0.000      -0.020      -0.012
tenure_squared           0.0199      0.004      4.813      0.000       0.012       0.028
age                      0.0521      0.007      7.198      0.000       0.038       0.066
age_squared             -0.1322      0.026     -5.153      0.000      -0.182      -0.082
years_educ               0.0210      0.007      2.912      0.004       0.007       0.035
commute_distance         0.0004      0.000      3.492      0.000       0.000       0.001
potential_experience     0.0010      0.000      3.287      0.001       0.000       0.002
mincer_residuals        -0.0728      0.020     -3.684      0.000      -0.112      -0.034
jobatrisk               -0.1430      0.019     -7.570      0.000      -0.180      -0.106
==============================================================================
Omnibus:                    16523.702   Durbin-Watson:                   1.846
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              378.759
Skew:                          -0.031   Prob(JB):                     5.67e-83
Kurtosis:                       1.704   Cond. No.                     9.79e+17
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The smallest eigenvalue is 5.12e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [45]:
df_OLS = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07)))
df_OLS["recog_sup"].value_counts()

C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:267: FutureWarning: Passing 'suffixes' which cause duplicate columns {'syear_x'} in the result is deprecated and will raise a MergeError in a future version.
  allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:220: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['gender'].replace('[2] Weiblich', 2,inplace=True)
C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

0.0    5524
1.0    2998
Name: recog_sup, dtype: int64

In [47]:
# optional argument ==1
df_card = include_jobchangers(add_mincer_residuals(merge_and_clean(df_05,df_06,df_07,work07)))
## drop n.a.ns
df_card.drop(columns=['similar_problems', 'take_revenge', 'insult_back','felt_recog_sup', 'felt_recog_effort',
       'recog_personal', 'felt_recog_personal', 'recog_pay', 'felt_recog_pay', 'year_birth', 'new_job',
       'reason_new_job', 'school_degree','overtime', 'recog_effort','wage_lastmonth','rec1' , 'rec2' , 'rec3' , 'rec4', 'rec5' , 'rec6' , 'rec7' ,'work_satisfaction'], inplace = True)
# add interaction term
df_card["recXrecog_sup"] = df_card["recog_sup"] * df_card["avg_rec"]
# drop missing data in regression dataframe
df_card.dropna(inplace=True)

import statsmodels.formula.api as smf

# Specify model
formula_main = 'turnover_intention ~ recog_sup + avg_rec + recXrecog_sup + working_hours + firmsize + tenure + tenure_squared + age + age_squared + years_educ + commute_distance + potential_experience + mincer_residuals + jobatrisk + sector'

# Fit the regression and cluster on the sector variable
reg_cardinal = smf.ols(formula_main, data=df_card).fit(cov_type='HC3')

# Print the regression results
summary = reg_cardinal.summary()
#print(summary.as_latex())
summary

C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:267: FutureWarning: Passing 'suffixes' which cause duplicate columns {'syear_x'} in the result is deprecated and will raise a MergeError in a future version.
  allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:220: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['gender'].replace('[2] Weiblich', 2,inplace=True)
C:\Users\max-admin\AppData\Local\Temp\ipykernel_17136\2983537223.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     turnover_intention   R-squared:                       0.219
Model:                            OLS   Adj. R-squared:                  0.208
Method:                 Least Squares   F-statistic:                     119.7
Date:                Thu, 19 Jan 2023   Prob (F-statistic):               0.00
Time:                        10:46:04   Log-Likelihood:                -3239.7
No. Observations:                5380   AIC:                             6621.
Df Residuals:                    5309   BIC:                             7089.
Df Model:                          70                                         
Covariance Type:                  HC3                                         
================================================================================================================================================================================================
                                                                                                                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                                                        0.0471      0.013      3.567      0.000       0.021       0.073
sector[T.[-2] trifft nicht zu]                                                                                                   0.0238      0.169      0.141      0.888      -0.308       0.356
sector[T.[10] Kohlenbergbau, Torfgewinnung]                                                                                     -0.0520      0.167     -0.311      0.756      -0.380       0.276
sector[T.[14] Gewinnung von Steinen und Erden, sonstiger Bergbau]                                                                0.4103      3.519      0.117      0.907      -6.488       7.308
sector[T.[15] Herstellung von Nahrungs- und Futtermitteln sowie Getränken]                                                      -0.0126      0.119     -0.106      0.916      -0.246       0.221
sector[T.[16] Tabakverarbeitung]                                                                                                -0.5369      0.110     -4.902      0.000      -0.752      -0.322
sector[T.[17] Herstellung von Textilien]                                                                                        -0.0358      0.144     -0.248      0.804      -0.318       0.247
sector[T.[18] Herstellung von Bekleidung]                                                                                        0.0082      0.164      0.050      0.960      -0.314       0.330
sector[T.[19] Herstellung von Leder und Lederwaren]                                                                             -0.3975      0.245     -1.622      0.105      -0.878       0.083
sector[T.[1] Landwirtschaft und  Jagd]                                                                                          -0.1519      0.132     -1.149      0.250      -0.411       0.107
sector[T.[20] Herstellung von Holz sowie Holz-, Kork- und Flechtwaren (ohne Herstellung von Möbeln)]                            -0.1087      0.152     -0.716      0.474      -0.406       0.189
sector[T.[21] Herstellung von Papier, Pappe und Waren daraus]                                                                   -0.2865      0.136     -2.101      0.036      -0.554      -0.019
sector[T.[22] Herstellung von Verlags- und Druckerzeugnissen,  Vervielfältigung von bespielten Ton-, Bild- und Datenträgern]     0.0427      0.120      0.357      0.721      -0.192       0